In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/books-4-languages/candide.txt
/kaggle/input/books-4-languages/zarathustra.txt
/kaggle/input/books-4-languages/quijote.txt
/kaggle/input/books-4-languages/wonderland.txt


In [2]:
def assess(expected, actual):
    if expected == actual:
        print("Pass")
    else:
        print(f"Fail; expected '{expected}', observed '{actual}'")

# Hidden Markov Models

This notebook uses the [hmmlearn](https://hmmlearn.readthedocs.io/en/latest/index.html) library to train a Hidden Markov Model (HMM).

We begin by including the `hmmlearn` library as well as a Python library for type annotations.

In [3]:
from hmmlearn import hmm
from typing import *
import time

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


To keep the model simple, we will preprocess our text inputs so that they only include lowercase letters and spaces. We will omit all punctuation and all whitespace characters that are not spaces. We will also add a leading space to each input. This process is encoded in the `filter_text()` function.

In [4]:
def filter_text(text: str) -> str:
    result = " "
    lowerbet = "qwertyuiopasdfghjklzxcvbnm "
    upperbet = "QWERTYUIOPASDFGHJKLZXCVBNM"
    for c in text:
        if c in lowerbet:
            result += c
        elif c in upperbet:
            for i in range(26):
                if upperbet[i] == c:
                    result += lowerbet[i]
        else:
            continue
    return result

In [5]:
assess(" this is a test",filter_text("This is a test."))
assess(" a long time ago in a galaxy far far away", filter_text("A long time ago, in a galaxy far, far away..."))

Pass
Pass


Next, we need to be able to build a vocabulary of all possible characters. We will write the `all_distinct_characters_from()` function in order to do so.

In [6]:
def all_distinct_characters_from(s: str) -> List[str]:
    result = []
    for c in s:
        if c not in result:
            result += c
    return result    

In [7]:
assess([' ', 'a', 'e', 'h', 'i', 's', 't'], sorted(all_distinct_characters_from("this is a test")))

Pass


The HMM will expect a sequence of integers rather than characters. We will use a list of characters (generated by the `all_distinct_characters_from()` function above) to transform a sequence of characters into a sequence of integers. The integers themselves need to be "wrapped" in lists, because tuples of integers are also acceptable emission symbols in the HMM.

We will write the `encode_sequence()` function to perform this task.

In [8]:
# (list of all chars in str) -> str -> [[only one int]]
def encode_sequence(reference_list: List[str], sequence: Iterable[str]) -> List[List[int]]:
    result = []
    for c in sequence:
        for i in range(len(reference_list)):
            if reference_list[i] == c:
                result += [[i]]
    return result        

In [9]:
assess([[4], [2], [0], [2], [4], [1], [3]], encode_sequence(['a', 'b', 'c', 'd', 'e'], 'ecacebd'))

Pass


To build an HMM, we [create a `hmm.CategoricalHMM()` object](https://hmmlearn.readthedocs.io/en/latest/api.html#hmmlearn.hmm.CategoricalHMM). Set the `num_components` parameter to the number of states and the `n_iter` parameter to 100. Next, call the `fit()` method with the provided sequence and assess it with the same sequence using the `score()` method. Then return both the model and the score.

A correct implementation of the `build_model_score()` function should be very short:
* Call the `hmm.CategoricalHMM()` constructor to build the model.
* Call `fit()` to train the model.
* Call `score()` to assess the model.

In [10]:
def build_model_score(num_states: int, training_sequence: List[int]) -> Tuple[hmm.CategoricalHMM, float]:
    model = hmm.CategoricalHMM(n_components=num_states, n_iter=100)
    model.fit(training_sequence)
    return model, model.score(training_sequence)

Note that each run will create a different model at random. The test ensures that it has the proper number of states, and it checks to see that the score is in a reasonable range.

In [11]:
model, score = build_model_score(2, [[4], [2], [0], [2], [4], [1], [3]])
if -10.0 <= score <= -6.0:
    print(f"Score {score} is reasonable")
else:
    print(f"Score {score} is not plausible")
if type(model) == hmm.CategoricalHMM:
    assess(model.n_components, 2)
else:
    print("Model is not of the hmm.CategoricalHMM data type.")

Score -9.360846767622009 is reasonable
Pass


Not all HMMs are created equal. Some models will perform better than others. In the `build_hmm_from()` function, we will perform the following operations to find a good HMM:
* Open the specified file and read it into a string.
* Call `all_distinct_characters_from()` to find all the characters employed by the string.
* Call `encode_sequence()` to encode the string as a list of lists of integers.
* Call `build_model_score()` `num_attempts` times. Keep the HMM with the largest score.
  * I suggest printing out the attempt number and the score for each attempt. This can
    take quite a while to run, and seeing the scores helps convey the pace of progress.
* Return the best model and the list of distinct characters from the input string.

In [12]:
def build_hmm_from(filename: str, num_states: int, num_attempts: int) -> Tuple[hmm.CategoricalHMM, List[str]]:
    book_path = os.path.join('/kaggle/input/books-4-languages/' + filename + '.txt')
    with open(book_path, 'r', encoding='utf-8') as book_file:
        book = filter_text(book_file.read())
    chars         = all_distinct_characters_from(book)
    trainSeq      = encode_sequence(chars,book)
    model, score  = build_model_score(num_states, trainSeq)
    for i in range(num_attempts-1):
        print(i+1)
        m, s  = build_model_score(num_states, trainSeq)
        if s > score:
            model = m
            score = s
    return model, chars 

In [13]:
build_hmm_from('wonderland',2,1)

(CategoricalHMM(n_components=2, n_features=27, n_iter=100,
                random_state=RandomState(MT19937) at 0x7B16743AF140),
 [' ',
  'p',
  'r',
  'o',
  'j',
  'e',
  'c',
  't',
  'g',
  'u',
  'n',
  'b',
  's',
  'a',
  'l',
  'i',
  'd',
  'v',
  'w',
  'y',
  'h',
  'k',
  'f',
  'm',
  'q',
  'x',
  'z'])

There is no specific unit test for the `build_hmm_from()` function - you will have to assess it below in terms of the results it produces.

Our methodology for understanding the meaning of the states of the HMM is as follows:
* Present a sequence of characters to the HMM.
* For each character, record which state is active.
* For each state, count how often each character is active in that state.
* Examining the character counts for each state, try to find a pattern that enables us to 
  say what the state represents.
  
The `state_histogram()` function takes two arguments: 
* A string representing the input to an HMM.
* A sequence of state numbers, one per character in the string.

The function returns a list of dictionaries, where each dictionary is a histogram of counts of characters that were observed in the given state.

In [14]:
def state_histogram(phrase: str, states: Sequence[int]) -> List[Dict[str,int]]:
    assert len(phrase) == len(states)
    dicts = [{}, {}]
    for i in range(len(states)):
        p = phrase[i]
        s = states[i]
        if p in dicts[s]:
            dicts[s][p] += 1
        else:
            dicts[s][p] = 1
    return dicts

In [15]:
assess([{' ': 4}, {'t': 3, 'h': 1, 'i': 2, 's': 3, 'a': 1, 'e': 1}], state_histogram(" this is a test", [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1]))

Pass


In the `state_histogram()` test above, state 0 corresponds to space characters, and state 1 corresponds to letters.

Now that we can construct a state histogram, we will write the `interpret_states()` function, which will give us an interpretation of the HMM as follows:
* Call `filter_text()` on the input phrase.
* Call the `predict()` method of `hmm.CategoricalHMM` to obtain a sequence of states corresponding to the letters in the filtered input fphrase. 
  * When calling `predict()`, use the `encode_sequence()` function to encode the phrase as a sequence of integers.
* Call `state_histogram()` and return a state histogram as described above.

In [19]:
def interpret_states(model: hmm.CategoricalHMM, observations: List[str], phrase: str) -> List[Dict[str,int]]:
    text = filter_text(phrase)
    states = model.predict(encode_sequence(observations, text))
    return state_histogram(text,states)

With `interpret_states()` completed, we can now train and interpret a full HMM. We will test it using the three English-language phrases below.

In [20]:
phrases = ["one does not simply walk into mordor",
           "stop trying to make fetch happen its not going to happen",
           "has anyone really been far even as decided to use even go want to do look more like"]

frases = ["uno no entra simplemente en mordor",
          "deja de intentar hacer que la búsqueda suceda, no va a suceder",
          "¿Alguien realmente ha estado tan lejos como para decidir usar incluso ir y querer parecerse más"]

cordes = ["on ne se contente pas d'entrer dans le mordor",
          "Arrêtez d'essayer de faire en sorte que la récupération se produise, cela n'arrivera pas",
          "Est-ce que quelqu'un a vraiment été aussi loin qu'il a décidé de l'utiliser, même s'il veut ressembler davantage à"]

satze  = ["Man geht nicht einfach nach Mordor",
          "Hör auf zu versuchen, den Abruf zu bewerkstelligen, das wird nicht passieren",
          "Ist irgendjemand wirklich so weit gekommen, dass er sich dazu entschlossen hat, sogar noch mehr auszusehen?"]

The experiment may take quite a while to build the model - my own solution required over two minutes. It will call `interpret_states()` for every phrase. If everything goes well, the meaning of the two states should be very clear.

In [22]:
# English Model - Alice In Wonderland

start = time.time()
model, observations = build_hmm_from('wonderland', 2, 10)
print(f"Finished building HMM after {time.time() - start} seconds.")
for phrase in phrases:
    print(phrase)
    print(interpret_states(model, observations, filter_text(phrase)))

1
2
3
4
5
6
7
8
9
Finished building HMM after 131.00361037254333 seconds.
one does not simply walk into mordor
[{' ': 8, 'o': 6, 'e': 2, 'i': 2, 'a': 1}, {'n': 3, 'd': 2, 's': 2, 't': 2, 'm': 2, 'p': 1, 'l': 2, 'y': 1, 'w': 1, 'k': 1, 'r': 2}]
stop trying to make fetch happen its not going to happen
[{' ': 12, 'o': 5, 'i': 3, 'a': 3, 'e': 4}, {'s': 2, 't': 7, 'p': 5, 'r': 1, 'y': 1, 'n': 5, 'g': 3, 'm': 1, 'k': 1, 'f': 1, 'c': 1, 'h': 3}]
has anyone really been far even as decided to use even go want to do look more like
[{' ': 19, 'a': 6, 'o': 8, 'e': 13, 'i': 2, 'u': 1}, {'h': 1, 's': 3, 'n': 6, 'y': 2, 'r': 3, 'l': 4, 'b': 1, 'f': 1, 'v': 2, 'd': 4, 'c': 1, 't': 3, 'g': 1, 'w': 1, 'k': 2, 'm': 1}]


In [23]:
# Spanish Model - Don Quixote

start = time.time()
model, observations = build_hmm_from('quijote', 2, 1)
print(f"Finished building HMM after {time.time() - start} seconds.")
for phrase in frases:
    print(phrase)
    print(interpret_states(model, observations, filter_text(phrase)))

Finished building HMM after 164.93392658233643 seconds.
uno no entra simplemente en mordor
[{' ': 7, 'n': 5, 't': 2, 'r': 3, 's': 1, 'm': 3, 'p': 1, 'l': 1, 'd': 1}, {'u': 1, 'o': 4, 'e': 5, 'a': 1, 'i': 1}]
deja de intentar hacer que la búsqueda suceda, no va a suceder
[{' ': 13, 'd': 5, 'j': 1, 'n': 3, 't': 2, 'r': 3, 'h': 1, 'c': 3, 'u': 2, 'l': 1, 'b': 1, 's': 3, 'v': 1}, {'e': 9, 'a': 8, 'i': 1, 'q': 2, 'u': 2, 'o': 1}]
¿Alguien realmente ha estado tan lejos como para decidir usar incluso ir y querer parecerse más
[{' ': 17, 'l': 4, 'g': 1, 'i': 1, 'n': 4, 'r': 9, 'm': 3, 't': 3, 'h': 1, 's': 6, 'd': 3, 'j': 1, 'c': 4, 'p': 2, 'u': 1}, {'a': 9, 'u': 3, 'e': 12, 'o': 5, 'i': 4, 'y': 1, 'q': 1}]


In [24]:
# French Model - Candide

start = time.time()
model, observations = build_hmm_from('candide', 2, 10)
print(f"Finished building HMM after {time.time() - start} seconds.")
for phrase in cordes:
    print(phrase)
    print(interpret_states(model, observations, filter_text(phrase)))

1
2
3
4
5
6
7
8
9
Finished building HMM after 144.74115347862244 seconds.
on ne se contente pas d'entrer dans le mordor
[{'n': 6, 's': 3, 'c': 1, 't': 3, 'p': 1, 'd': 3, 'r': 4, 'l': 1, 'm': 1}, {' ': 10, 'o': 4, 'e': 7, 'a': 2}]
Arrêtez d'essayer de faire en sorte que la récupération se produise, cela n'arrivera pas
[{'r': 11, 't': 3, 'z': 1, 'd': 3, 's': 6, 'y': 1, 'f': 1, 'n': 3, 'q': 1, 'l': 2, 'c': 2, 'p': 3, 'v': 1}, {' ': 15, 'a': 9, 'e': 12, 'i': 4, 'o': 3, 'u': 3}]
Est-ce que quelqu'un a vraiment été aussi loin qu'il a décidé de l'utiliser, même s'il veut ressembler davantage à
[{'s': 7, 't': 6, 'c': 2, 'q': 4, 'l': 7, 'n': 4, 'v': 3, 'r': 4, 'm': 4, 'd': 4, 'b': 1, 'g': 1}, {' ': 20, 'e': 13, 'u': 8, 'a': 7, 'i': 8, 'o': 1}]


In [25]:
# German Model - Thus Spoke Zarathustra

start = time.time()
model, observations = build_hmm_from('zarathustra', 2, 10)
print(f"Finished building HMM after {time.time() - start} seconds.")
for phrase in satze:
    print(phrase)
    print(interpret_states(model, observations, filter_text(phrase)))

1
2
3
4
5
6
7
8
9
Finished building HMM after 375.71434259414673 seconds.
Man geht nicht einfach nach Mordor
[{' ': 7, 'a': 3, 'e': 2, 'i': 2, 'c': 3, 'o': 2}, {'m': 2, 'n': 4, 'g': 1, 'h': 4, 't': 2, 'f': 1, 'r': 2, 'd': 1}]
Hör auf zu versuchen, den Abruf zu bewerkstelligen, das wird nicht passieren
[{' ': 13, 'a': 4, 'u': 5, 'e': 9, 'c': 2, 'k': 1, 'i': 4, 'p': 1}, {'h': 3, 'r': 6, 'f': 2, 'z': 2, 'v': 1, 's': 5, 'n': 5, 'd': 3, 'b': 2, 'w': 2, 't': 2, 'l': 2, 'g': 1}]
Ist irgendjemand wirklich so weit gekommen, dass er sich dazu entschlossen hat, sogar noch mehr auszusehen?
[{' ': 17, 'i': 6, 'e': 11, 'a': 6, 'k': 1, 'c': 4, 'o': 5, 'u': 3, 't': 1}, {'s': 11, 't': 3, 'r': 5, 'g': 3, 'n': 7, 'd': 4, 'j': 1, 'm': 4, 'w': 2, 'l': 2, 'h': 7, 'k': 1, 'z': 2}]
